In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
from jupyterthemes import jtplot
jtplot.reset()
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Output signal trend

***

## Arduino accuracy

In [2]:
err = 3.3 * 4 * 0.58 / 4096 #volt
err

0.0018691406249999997

## Input signal

In [3]:
Vin = 1.018 #volt
errVin = 0.019 #volt

Vin_th = 1 #volt

## Preamp output signal

In [4]:
Vpre = 0.3862829236546532 #volt
errVpre = err #volt

Vpre_th = 0.3878731202 #volt

## Shaper (ideal) output signal

In [5]:
Vsh_ideal = 0.34153256778309415 #volt
errVsh_ideal = err #volt

Vsh_ideal_th = 0.3673765 #volt

## Shaper (real - no Rpz) output signal

In [6]:
Vsh_norpz = 0.11445256778309399 #volt
errVsh_norpz = err #volt

Vsh_norpz_th = 0.14901910000000002 #volt

## Shaper (real - Rpz) output signal

In [7]:
Vsh_rpz = 0.131182567783094 #volt
errVsh_rpz = err #volt

Vsh_rpz_th = 0.1458615 #volt

## Chain output signal

In [8]:
Vchain = 1.2711323763955342 # volt
errVchain = err # volt

Vchain_th = Vsh_rpz_th * 9.254157483 #volt

## Output signal trend plot

### Arrange Data

In [9]:
d1 = {
    'Stage' : ['IN', 'Shaper'], 
    'Vmax' : [Vin, Vsh_ideal], 
    'Vmax th' : [Vin_th, Vsh_ideal_th]
}

d2 = {
    'Stage' : ['IN', 'PreAmp', 'Shaper'], 
    'Vmax' : [Vin, Vpre, Vsh_norpz], 
    'Vmax th' : [Vin_th, Vpre_th, Vsh_norpz_th]
}

d3 = {
    'Stage' : ['IN', 'PreAmp', 'Shaper', 'OUT'], 
    'Vmax' : [Vin, Vpre, Vsh_rpz, Vchain], 
    'Vmax th' : [Vin_th, Vpre_th, Vsh_rpz_th, Vchain_th]
}

data1 = pd.DataFrame(data=d1)
data2 = pd.DataFrame(data=d2)
data3 = pd.DataFrame(data=d3)

data11 = pd.Categorical(data1['Stage'], ordered = True, categories=['IN', 'Shaper'])
data22 = pd.Categorical(data2['Stage'], ordered = True, categories=['IN', 'PreAmp', 'Shaper'])
data33 = pd.Categorical(data3['Stage'], ordered = True, categories=['IN', 'PreAmp', 'Shaper', 'OUT'])

In [10]:
d = {
    'Stage' : ['IN', 'PreAmp','Shaper', 'IN', 'PreAmp', 'Shaper', 'IN', 'PreAmp', 'Shaper', 'OUT'],
    'Vmax' : [Vin, np.NaN, Vsh_ideal, Vin, Vpre, Vsh_norpz, Vin, Vpre, Vsh_rpz, Vchain],
    'Configuration' : ['Ideal', 'Ideal', 'Ideal', 'Real w/o Rpz', 'Real w/o Rpz', 'Real w/o Rpz', 'Real w/ Rpz', 'Real w/ Rpz', 'Real w/ Rpz', 'Real w/ Rpz']
}

data = pd.DataFrame(data=d)

Labels = ['IN', 'PreAmp', 'Shaper', 'OUT']

Palette = ['#02CB00', '#FF4B00', '#227FF7']

In [11]:
dd = {
    'Stage' : ['IN', 'PreAmp', 'Shaper', 'OUT', 'IN', 'PreAmp', 'Shaper', 'OUT'],
    'Vmax' : [Vin, Vpre, Vsh_rpz, Vchain, Vin_th, Vpre_th, Vsh_rpz_th, Vchain_th],
    'Config' : ['Experimental', 'Experimental', 'Experimental', 'Experimental', 'Theoretical', 'Theoretical', 'Theoretical', 'Theoretical']
}

ddata = pd.DataFrame(data=dd)
Labelss = ['IN', 'PreAmp', 'Shaper', 'OUT']
Palettee = ['#FF4B00', '#227FF7']

In [12]:
fig = plt.figure(figsize=(25,10))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

fig.suptitle('Output Signal Trend', fontsize = 36)



ax1.plot([-0.03, 1.95], [Vin, Vsh_ideal], color = '#02CB00', lw = 5.5, alpha = 0.7, zorder = 0)

g = sns.pointplot(x = 'Stage', y = 'Vmax', hue = 'Configuration', data = data, order = Labels, join=True, dodge = True, palette=Palette, scale = 2, plot_kws=dict(alpha=0.7), ax = ax1)

plt.setp(g.collections, alpha=.7) 
plt.setp(g.lines, alpha=.7)  


#plt.setp(ax1.get_legend().get_texts(), fontsize='24') 
#plt.setp(ax1.get_legend().get_title(), fontsize='32')


ax1.tick_params(axis = 'both', which = 'major', labelsize = 26, direction = 'in', length = 10)
ax1.tick_params(axis = 'y', which = 'minor', labelsize = 26, direction = 'in', length = 5)
ax1.tick_params(axis = 'x', which = 'minor', labelsize = 26, direction = 'in', length = 0)
ax1.set_yticks(ticks = ax1.get_yticks(), minor = True)
ax1.minorticks_on()

ax1.set_xlabel('', fontsize = 0, loc = 'center')
ax1.set_ylabel('Max Output Signal (V)', fontsize = 32, loc = 'center')

ax1.set_title('Shaper Configurations', fontsize = 32)

ax1.set_xlim(left = -0.2, right = 3.2)
ax1.set_ylim(bottom = 0, top = 1.4)

ax1.legend(loc = 'best', prop = {'size': 24}, ncol = 1, frameon = True, fancybox = False, framealpha = 1)


g1 = sns.pointplot(x = 'Stage', y = 'Vmax', hue = 'Config', data = ddata, order = Labelss, hue_order = ['Theoretical', 'Experimental'], join=True, dodge = True, palette=Palettee, scale = 2, plot_kws=dict(alpha=0.7), ax = ax2)

plt.setp(g1.collections, alpha=.7) 
plt.setp(g1.lines, alpha=.7)  

ax2.tick_params(axis = 'both', which = 'major', labelsize = 26, direction = 'in', length = 10)
ax2.tick_params(axis = 'y', which = 'minor', labelsize = 26, direction = 'in', length = 5)
ax2.tick_params(axis = 'x', which = 'minor', labelsize = 26, direction = 'in', length = 0)
ax2.set_yticks(ticks = ax1.get_yticks(), minor = True)
ax2.minorticks_on()

ax2.set_xlabel('', fontsize = 0, loc = 'center')
ax2.set_ylabel('Max Output Signal (V)', fontsize = 32, loc = 'center')

ax2.set_title('Theoretical Comparison', fontsize = 32)

ax2.set_xlim(left = -0.2, right = 3.2)
ax2.set_ylim(bottom = 0, top = 1.4)

ax2.legend(loc = 'best', prop = {'size': 24}, ncol = 1, frameon = True, fancybox = False, framealpha = 1)

#fig.savefig('../Plots/Catena/trend3.png', dpi = 300, facecolor='white')

plt.show()

In [16]:
fig = plt.figure(figsize=(12,7))
ax1 = fig.add_subplot(1, 1, 1)
ax1.set_axisbelow(True)
ax1.grid(b = True, which='major', axis='y', zorder = 1, color = '#000000', linewidth = 0.3)


g1 = sns.pointplot(x = 'Stage', y = 'Vmax', hue = 'Config', data = ddata, order = Labelss, hue_order = ['Theoretical', 'Experimental'], join=True, dodge = True, palette=Palettee, scale = 2, ax = ax1, zorder=2)

ax1.tick_params(axis = 'both', which = 'major', labelsize = 26, direction = 'in', length = 10)
ax1.tick_params(axis = 'y', which = 'minor', labelsize = 26, direction = 'in', length = 5)
ax1.tick_params(axis = 'x', which = 'minor', labelsize = 26, direction = 'in', length = 0)
ax1.set_yticks(ticks = ax1.get_yticks(), minor = True)
ax1.minorticks_on()

ax1.set_xlabel('', fontsize = 0, loc = 'center')
ax1.set_ylabel('Signal Amplitude (V)', fontsize = 32, loc = 'center')

ax1.set_title('Output Signal Trend', fontsize = 36)

ax1.set_xlim(left = -0.2, right = 3.2)
ax1.set_ylim(bottom = 0, top = 1.4)

ax1.legend(loc = 'best', prop = {'size': 24}, ncol = 1, frameon = True, fancybox = False, framealpha = 1)

fig.savefig('../Plots/Catena/trend4.png', dpi = 300, facecolor='white')

plt.show()